We are going to use a stacy's model which scored given headlines, and run a logistic regression to test weather or not the model is able to predict which way the stock market moved the current day and/or following day. 

In [74]:
import pandas_datareader as web
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date

In [75]:
#First we bring in the combined data set - which has the scores of each headline that the LDA model generated. 
df = pd.read_csv('../Data/Combined_News_DJIA_topics.csv')
df.head()

,Unnamed: 0,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,...,Top16_topic_score,Top17_topic_score,Top18_topic_score,Top19_topic_score,Top20_topic_score,Top21_topic_score,Top22_topic_score,Top23_topic_score,Top24_topic_score,Top25_topic_score
0,0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...",...,"[5, 0.6031984]","[2, 0.47149044]","[3, 0.36061332]","[5, 0.45405966]","[5, 0.5703015]","[5, 0.54791677]","[5, 0.49842814]","[1, 0.8499608]","[6, 0.43135294]","[1, 0.81994975]"
1,1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,...,"[9, 0.53071535]","[9, 0.611838]","[7, 0.5506175]","[6, 0.81994605]","[0, 0.57922643]","[4, 0.6275869]","[4, 0.6561521]","[5, 0.51787627]","[6, 0.37684822]","[6, 0.7749493]"
2,2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,...,"[5, 0.8499421]","[5, 0.6254238]","[8, 0.44398957]","[5, 0.8199832]","[1, 0.52361417]","[5, 0.7748322]","[5, 0.573867]","[5, 0.28484675]","[1, 0.69997424]","[6, 0.8499552]"
3,3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",...,"[2, 0.69973093]","[5, 0.35203508]","[3, 0.48795778]","[3, 0.5979013]","[5, 0.46311668]","[5, 0.3666421]","[3, 0.5055619]","[8, 0.3956947]","[3, 0.71894366]","[3, 0.6346879]"
4,4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,...,"[6, 0.5339442]","[5, 0.49618754]","[3, 0.43610215]","[5, 0.42683432]","[9, 0.5254859]","[8, 0.88747185]","[7, 0.574972]","[8, 0.4857589]","[2, 0.46074417]","[8, 0.6320944]"


In [76]:
#Next clean all those strings of scores.... and map the scores to a new column. 
df["new1"] = df["Top1_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score1"] = df.new1.map(lambda x: x[0])

df["new2"] = df["Top2_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score2"] = df.new2.map(lambda x: x[0])

df["new3"] = df["Top3_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score3"] = df.new3.map(lambda x: x[0])

df["new4"] = df["Top4_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score4"] = df.new4.map(lambda x: x[0])

df["new5"] = df["Top5_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score5"] = df.new5.map(lambda x: x[0])

df["new6"] = df["Top6_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score6"] = df.new6.map(lambda x: x[0])

df["new7"] = df["Top7_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score7"] = df.new7.map(lambda x: x[0])

df["new8"] = df["Top8_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score8"] = df.new8.map(lambda x: x[0])

df["new9"] = df["Top9_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score9"] = df.new9.map(lambda x: x[0])

df["new10"] = df["Top10_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score10"] = df.new10.map(lambda x: x[0])

df["new11"] = df["Top11_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score11"] = df.new11.map(lambda x: x[0])

df["new12"] = df["Top12_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score12"] = df.new12.map(lambda x: x[0])

df["new13"] = df["Top13_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score13"] = df.new13.map(lambda x: x[0])

df["new14"] = df["Top14_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score14"] = df.new14.map(lambda x: x[0])

df["new15"] = df["Top15_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score15"] = df.new15.map(lambda x: x[0])

df["new16"] = df["Top16_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score16"] = df.new16.map(lambda x: x[0])

df["new17"] = df["Top17_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score17"] = df.new17.map(lambda x: x[0])

df["new18"] = df["Top18_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score18"] = df.new18.map(lambda x: x[0])

df["new19"] = df["Top19_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score19"] = df.new19.map(lambda x: x[0])

df["new20"] = df["Top20_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score20"] = df.new20.map(lambda x: x[0])

df["new21"] = df["Top21_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score21"] = df.new21.map(lambda x: x[0])

df["new22"] = df["Top22_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score22"] = df.new22.map(lambda x: x[0])

df["new23"] = df["Top23_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score23"] = df.new23.map(lambda x: x[0])

df["new24"] = df["Top24_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score24"] = df.new24.map(lambda x: x[0])

df["new25"] = df["Top25_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score25"] = df.new25.map(lambda x: x[0])
df.columns

Index(['Unnamed: 0', 'Date', 'Label', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5',
       'Top6', 'Top7',
       ...
       'new21', 'score21', 'new22', 'score22', 'new23', 'score23', 'new24',
       'score24', 'new25', 'score25'],
      dtype='object', length=103)

In [78]:
#change all those new columns to integers/decimals.
#this is what we'll need for the logistic regression along with the "Label" column
df["score1"] = pd.to_numeric(df["score1"], downcast="float")
df["score2"] = pd.to_numeric(df["score2"], downcast="float")
df["score3"] = pd.to_numeric(df["score3"], downcast="float")
df["score4"] = pd.to_numeric(df["score4"], downcast="float")
df["score5"] = pd.to_numeric(df["score5"], downcast="float")
df["score6"] = pd.to_numeric(df["score6"], downcast="float")
df["score7"] = pd.to_numeric(df["score7"], downcast="float")
df["score8"] = pd.to_numeric(df["score8"], downcast="float")
df["score9"] = pd.to_numeric(df["score9"], downcast="float")
df["score10"] = pd.to_numeric(df["score10"], downcast="float")
df["score11"] = pd.to_numeric(df["score11"], downcast="float")
df["score12"] = pd.to_numeric(df["score12"], downcast="float")
df["score13"] = pd.to_numeric(df["score13"], downcast="float")
df["score14"] = pd.to_numeric(df["score14"], downcast="float")
df["score15"] = pd.to_numeric(df["score15"], downcast="float")
df["score16"] = pd.to_numeric(df["score16"], downcast="float")
df["score17"] = pd.to_numeric(df["score17"], downcast="float")
df["score18"] = pd.to_numeric(df["score18"], downcast="float")
df["score19"] = pd.to_numeric(df["score19"], downcast="float")
df["score20"] = pd.to_numeric(df["score20"], downcast="float")
df["score21"] = pd.to_numeric(df["score21"], downcast="float")
df["score22"] = pd.to_numeric(df["score22"], downcast="float")
df["score23"] = pd.to_numeric(df["score23"], downcast="float")
df["score24"] = pd.to_numeric(df["score24"], downcast="float")
df["score25"] = pd.to_numeric(df["score25"], downcast="float")
df["Label"] = pd.to_numeric(df["Label"], downcast="float")


In [79]:
#time to do logistic regression
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [83]:
# Assign X (data) and y (target)
X = df[["score1", "score2", "score3", "score4", "score5", "score6", "score7", "score8", "score9", "score10",\
       "score11", "score12", "score13", "score14", "score15", "score16", "score17", "score18", "score19",\
       "score20", "score21", "score22", "score23", "score24", "score25"]]
y = df["Label"]
print(X.shape, y.shape)

(1986, 25) (1986,)


In [84]:
#import dependencies and run train test split on the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [85]:
#set up a logistic regression classifer
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
classifier

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
#fit the training data to the model
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [87]:
#compare the score of training to testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5486903962390867
Testing Data Score: 0.49899396378269617


In [89]:
#make predictions on the testing data and compare to results of actual data. 
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]
First 10 Actual labels: [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]


In [96]:
#Display results of the testing data and the predictions
resultsDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
resultsDF["correct"] = np.where(resultsDF['Prediction'] == resultsDF["Actual"], "Yes", "No")
resultsDF

,Prediction,Actual,correct
0,0.0,0.0,Yes
1,0.0,0.0,Yes
2,0.0,1.0,No
3,1.0,1.0,Yes
4,0.0,1.0,No
...,...,...,...
492,1.0,0.0,No
493,1.0,0.0,No
494,1.0,1.0,Yes
495,0.0,0.0,Yes


In [97]:
#how many right? how many wrong? 
resultsDF = resultsDF.groupby("correct").count()
resultsDF = resultsDF["Prediction"]
resultsDF

correct
No     249
Yes    248
Name: Prediction, dtype: int64

In [129]:
#clearly the scores alone didn't capture a very good model to predict the moves of the market on those days. 
#let's try predicting next day... just will have to adjust the data a bit, then run the same steps. 
df = pd.read_csv('../Data/Combined_News_DJIA_topics.csv')
df.head()

,Unnamed: 0,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,...,Top16_topic_score,Top17_topic_score,Top18_topic_score,Top19_topic_score,Top20_topic_score,Top21_topic_score,Top22_topic_score,Top23_topic_score,Top24_topic_score,Top25_topic_score
0,0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...",...,"[5, 0.6031984]","[2, 0.47149044]","[3, 0.36061332]","[5, 0.45405966]","[5, 0.5703015]","[5, 0.54791677]","[5, 0.49842814]","[1, 0.8499608]","[6, 0.43135294]","[1, 0.81994975]"
1,1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,...,"[9, 0.53071535]","[9, 0.611838]","[7, 0.5506175]","[6, 0.81994605]","[0, 0.57922643]","[4, 0.6275869]","[4, 0.6561521]","[5, 0.51787627]","[6, 0.37684822]","[6, 0.7749493]"
2,2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,...,"[5, 0.8499421]","[5, 0.6254238]","[8, 0.44398957]","[5, 0.8199832]","[1, 0.52361417]","[5, 0.7748322]","[5, 0.573867]","[5, 0.28484675]","[1, 0.69997424]","[6, 0.8499552]"
3,3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",...,"[2, 0.69973093]","[5, 0.35203508]","[3, 0.48795778]","[3, 0.5979013]","[5, 0.46311668]","[5, 0.3666421]","[3, 0.5055619]","[8, 0.3956947]","[3, 0.71894366]","[3, 0.6346879]"
4,4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,...,"[6, 0.5339442]","[5, 0.49618754]","[3, 0.43610215]","[5, 0.42683432]","[9, 0.5254859]","[8, 0.88747185]","[7, 0.574972]","[8, 0.4857589]","[2, 0.46074417]","[8, 0.6320944]"


In [130]:
#make sure sorted by date, then move up label
df["Actual_Date"] = pd.to_datetime(df["Date"], format='%Y-%m-%d', errors='ignore')
df = df.sort_values(by="Actual_Date", ascending=True)
df["Label"] = df["Label"].shift(-1)
df.drop(df.tail(2).index,inplace=True)

In [131]:
#Next clean all those strings of scores.... and map the scores to a new column. 
df["new1"] = df["Top1_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score1"] = df.new1.map(lambda x: x[0])

df["new2"] = df["Top2_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score2"] = df.new2.map(lambda x: x[0])

df["new3"] = df["Top3_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score3"] = df.new3.map(lambda x: x[0])

df["new4"] = df["Top4_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score4"] = df.new4.map(lambda x: x[0])

df["new5"] = df["Top5_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score5"] = df.new5.map(lambda x: x[0])

df["new6"] = df["Top6_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score6"] = df.new6.map(lambda x: x[0])

df["new7"] = df["Top7_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score7"] = df.new7.map(lambda x: x[0])

df["new8"] = df["Top8_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score8"] = df.new8.map(lambda x: x[0])

df["new9"] = df["Top9_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score9"] = df.new9.map(lambda x: x[0])

df["new10"] = df["Top10_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score10"] = df.new10.map(lambda x: x[0])

df["new11"] = df["Top11_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score11"] = df.new11.map(lambda x: x[0])

df["new12"] = df["Top12_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score12"] = df.new12.map(lambda x: x[0])

df["new13"] = df["Top13_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score13"] = df.new13.map(lambda x: x[0])

df["new14"] = df["Top14_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score14"] = df.new14.map(lambda x: x[0])

df["new15"] = df["Top15_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score15"] = df.new15.map(lambda x: x[0])

df["new16"] = df["Top16_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score16"] = df.new16.map(lambda x: x[0])

df["new17"] = df["Top17_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score17"] = df.new17.map(lambda x: x[0])

df["new18"] = df["Top18_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score18"] = df.new18.map(lambda x: x[0])

df["new19"] = df["Top19_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score19"] = df.new19.map(lambda x: x[0])

df["new20"] = df["Top20_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score20"] = df.new20.map(lambda x: x[0])

df["new21"] = df["Top21_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score21"] = df.new21.map(lambda x: x[0])

df["new22"] = df["Top22_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score22"] = df.new22.map(lambda x: x[0])

df["new23"] = df["Top23_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score23"] = df.new23.map(lambda x: x[0])

df["new24"] = df["Top24_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score24"] = df.new24.map(lambda x: x[0])

df["new25"] = df["Top25_topic_score"].str.replace("[","").str.replace(",","").str.replace("]","").str.split(" ")
df.loc[:, "score25"] = df.new25.map(lambda x: x[0])
df.columns

Index(['Unnamed: 0', 'Date', 'Label', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5',
       'Top6', 'Top7',
       ...
       'new21', 'score21', 'new22', 'score22', 'new23', 'score23', 'new24',
       'score24', 'new25', 'score25'],
      dtype='object', length=104)

In [132]:
#change all those new columns to integers/decimals.
#this is what we'll need for the logistic regression along with the "Label" column
df["score1"] = pd.to_numeric(df["score1"], downcast="float")
df["score2"] = pd.to_numeric(df["score2"], downcast="float")
df["score3"] = pd.to_numeric(df["score3"], downcast="float")
df["score4"] = pd.to_numeric(df["score4"], downcast="float")
df["score5"] = pd.to_numeric(df["score5"], downcast="float")
df["score6"] = pd.to_numeric(df["score6"], downcast="float")
df["score7"] = pd.to_numeric(df["score7"], downcast="float")
df["score8"] = pd.to_numeric(df["score8"], downcast="float")
df["score9"] = pd.to_numeric(df["score9"], downcast="float")
df["score10"] = pd.to_numeric(df["score10"], downcast="float")
df["score11"] = pd.to_numeric(df["score11"], downcast="float")
df["score12"] = pd.to_numeric(df["score12"], downcast="float")
df["score13"] = pd.to_numeric(df["score13"], downcast="float")
df["score14"] = pd.to_numeric(df["score14"], downcast="float")
df["score15"] = pd.to_numeric(df["score15"], downcast="float")
df["score16"] = pd.to_numeric(df["score16"], downcast="float")
df["score17"] = pd.to_numeric(df["score17"], downcast="float")
df["score18"] = pd.to_numeric(df["score18"], downcast="float")
df["score19"] = pd.to_numeric(df["score19"], downcast="float")
df["score20"] = pd.to_numeric(df["score20"], downcast="float")
df["score21"] = pd.to_numeric(df["score21"], downcast="float")
df["score22"] = pd.to_numeric(df["score22"], downcast="float")
df["score23"] = pd.to_numeric(df["score23"], downcast="float")
df["score24"] = pd.to_numeric(df["score24"], downcast="float")
df["score25"] = pd.to_numeric(df["score25"], downcast="float")
df["Label"] = pd.to_numeric(df["Label"], downcast="float")


In [133]:
#time to do logistic regression
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [134]:
# Assign X (data) and y (target)
X = df[["score1", "score2", "score3", "score4", "score5", "score6", "score7", "score8", "score9", "score10",\
       "score11", "score12", "score13", "score14", "score15", "score16", "score17", "score18", "score19",\
       "score20", "score21", "score22", "score23", "score24", "score25"]]
y = df["Label"]
print(X.shape, y.shape)

(1984, 25) (1984,)


In [135]:
#import dependencies and run train test split on the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [136]:
#set up a logistic regression classifer
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
classifier

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [137]:
#fit the training data to the model
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [138]:
#compare the score of training to testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5611559139784946
Testing Data Score: 0.5040322580645161


In [139]:
#make predictions on the testing data and compare to results of actual data. 
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]
First 10 Actual labels: [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]


In [140]:
#Display results of the testing data and the predictions
resultsDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
resultsDF["correct"] = np.where(resultsDF['Prediction'] == resultsDF["Actual"], "Yes", "No")
resultsDF

,Prediction,Actual,correct
0,1.0,0.0,No
1,1.0,1.0,Yes
2,0.0,0.0,Yes
3,1.0,0.0,No
4,1.0,1.0,Yes
...,...,...,...
491,0.0,0.0,Yes
492,1.0,0.0,No
493,1.0,0.0,No
494,0.0,0.0,Yes


In [141]:
#how many right? how many wrong? 
resultsDF = resultsDF.groupby("correct").count()
resultsDF = resultsDF["Prediction"]
resultsDF

correct
No     246
Yes    250
Name: Prediction, dtype: int64

In [1]:
#This part we will try to run logistic analysis again, but on the refined data from Stacy's LDA Topics
import pandas_datareader as web
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date

In [2]:
#First we bring in the combined data set - which has the scores of each headline that the LDA model generated. 
df = pd.read_csv('../Data/Combined_News_DJIA_single_topic.csv')
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_headlines,daily_topic,daily_words
0,8/8/08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",Georgia 'downs two Russian warplanes' as coun...,3,"0.006*""right"" + 0.005*""russia"" + 0.004*""isis"" ..."
1,8/11/08,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",Why wont America and Nato help us? If they wo...,5,"0.009*""russia"" + 0.007*""russian"" + 0.005*""ukra..."
2,8/12/08,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,"b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",Remember that adorable 9-year-old who sang at...,3,"0.006*""right"" + 0.005*""russia"" + 0.004*""isis"" ..."
3,8/13/08,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,"b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,U.S. refuses Israel weapons to attack Iran: ...,5,"0.009*""russia"" + 0.007*""russian"" + 0.005*""ukra..."
4,8/14/08,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,"b'""Non-media"" photos of South Osse

In [8]:
#time to do logistic regression
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [17]:
# Assign X (data) and y (target)
X = df[["daily_topic"]]
y = df["Label"]
print(X.shape, y.shape)

(1986, 1) (1986,)


In [18]:
#import dependencies and run train test split on the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [19]:
#set up a logistic regression classifer
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
classifier

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
#fit the training data to the model
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
#compare the score of training to testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5278710543989255
Testing Data Score: 0.5372233400402414


In [22]:
#make predictions on the testing data and compare to results of actual data. 
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0, 1, 1, 0, 1, 1, 1, 1, 0, 1]
First 10 Actual labels: [0, 0, 1, 1, 1, 1, 1, 1, 0, 1]


In [23]:
#Display results of the testing data and the predictions
resultsDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
resultsDF["correct"] = np.where(resultsDF['Prediction'] == resultsDF["Actual"], "Yes", "No")
resultsDF

,Prediction,Actual,correct
0,0,0,Yes
1,1,0,No
2,1,1,Yes
3,0,1,No
4,1,1,Yes
...,...,...,...
492,1,0,No
493,0,0,Yes
494,1,1,Yes
495,1,0,No


In [24]:
#how many right? how many wrong? 
resultsDF = resultsDF.groupby("correct").count()
resultsDF = resultsDF["Prediction"]
resultsDF

correct
No     230
Yes    267
Name: Prediction, dtype: int64

In [25]:
#let's try predicting next day... just will have to adjust the data a bit, then run the same steps. 
df = pd.read_csv('../Data/Combined_News_DJIA_single_topic.csv')
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_headlines,daily_topic,daily_words
0,8/8/08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",Georgia 'downs two Russian warplanes' as coun...,3,"0.006*""right"" + 0.005*""russia"" + 0.004*""isis"" ..."
1,8/11/08,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",Why wont America and Nato help us? If they wo...,5,"0.009*""russia"" + 0.007*""russian"" + 0.005*""ukra..."
2,8/12/08,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,"b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",Remember that adorable 9-year-old who sang at...,3,"0.006*""right"" + 0.005*""russia"" + 0.004*""isis"" ..."
3,8/13/08,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,"b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,U.S. refuses Israel weapons to attack Iran: ...,5,"0.009*""russia"" + 0.007*""russian"" + 0.005*""ukra..."
4,8/14/08,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,"b'""Non-media"" photos of South Osse

In [26]:
#make sure sorted by date, then move up label
df["Actual_Date"] = pd.to_datetime(df["Date"], format='%Y-%m-%d', errors='ignore')
df = df.sort_values(by="Actual_Date", ascending=True)
df["Label"] = df["Label"].shift(-1)
df.drop(df.tail(2).index,inplace=True)

In [27]:
#time to do logistic regression
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [28]:
# Assign X (data) and y (target)
X = df[["daily_topic"]]
y = df["Label"]
print(X.shape, y.shape)

(1984, 1) (1984,)


In [29]:
#import dependencies and run train test split on the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [30]:
#set up a logistic regression classifer
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
classifier

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
#fit the training data to the model
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
#compare the score of training to testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5168010752688172
Testing Data Score: 0.5040322580645161


In [33]:
#make predictions on the testing data and compare to results of actual data. 
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0]
First 10 Actual labels: [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]


In [34]:
#Display results of the testing data and the predictions
resultsDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
resultsDF["correct"] = np.where(resultsDF['Prediction'] == resultsDF["Actual"], "Yes", "No")
resultsDF

,Prediction,Actual,correct
0,0.0,0.0,Yes
1,1.0,0.0,No
2,0.0,0.0,Yes
3,1.0,1.0,Yes
4,0.0,1.0,No
...,...,...,...
491,1.0,1.0,Yes
492,1.0,1.0,Yes
493,0.0,1.0,No
494,1.0,1.0,Yes


In [35]:
#how many right? how many wrong? 
resultsDF = resultsDF.groupby("correct").count()
resultsDF = resultsDF["Prediction"]
resultsDF

correct
No     246
Yes    250
Name: Prediction, dtype: int64

In [36]:
#Now will try the sentiment analysis...
df = pd.read_csv('../Data/sentiment_values.csv')
df.head()

,Unnamed: 0,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,...,Top17sentiment_value,Top18sentiment_value,Top19sentiment_value,Top20sentiment_value,Top21sentiment_value,Top22sentiment_value,Top23sentiment_value,Top24sentiment_value,Top25sentiment_value,Daily_Sentiment
0,0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...",...,0.0,0.2,0.2,0.0,-0.8,-0.6,-0.3,-0.9,0.4,-0.30
1,1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,...,0.0,-0.7,0.0,0.0,-0.8,0.0,-0.3,-0.2,0.0,-0.08
2,2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,...,0.0,0.1,-0.6,-0.6,0.0,0.5,0.4,0.0,0.0,-0.22
3,3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",...,0.0,0.0,-0.5,-0.0,-0.3,0.5,-0.6,-0.4,-0.3,-0.15
4,4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,...,0.0,0.0,0.0,0.0,-0.4,-0.6,0.2,-0.7,0.7,-0.08


In [41]:
#time to do logistic regression
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [42]:
df.columns

Index(['Unnamed: 0', 'Date', 'Label', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5',
       'Top6', 'Top7', 'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13',
       'Top14', 'Top15', 'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21',
       'Top22', 'Top23', 'Top24', 'Top25', 'Top1sentiment_value',
       'Top2sentiment_value', 'Top3sentiment_value', 'Top4sentiment_value',
       'Top5sentiment_value', 'Top6sentiment_value', 'Top7sentiment_value',
       'Top8sentiment_value', 'Top9sentiment_value', 'Top10sentiment_value',
       'Top11sentiment_value', 'Top12sentiment_value', 'Top13sentiment_value',
       'Top14sentiment_value', 'Top15sentiment_value', 'Top16sentiment_value',
       'Top17sentiment_value', 'Top18sentiment_value', 'Top19sentiment_value',
       'Top20sentiment_value', 'Top21sentiment_value', 'Top22sentiment_value',
       'Top23sentiment_value', 'Top24sentiment_value', 'Top25sentiment_value',
       'Daily_Sentiment'],
      dtype='object')

In [43]:
# Assign X (data) and y (target)
X = df[['Top1sentiment_value',
       'Top2sentiment_value', 'Top3sentiment_value', 'Top4sentiment_value',
       'Top5sentiment_value', 'Top6sentiment_value', 'Top7sentiment_value',
       'Top8sentiment_value', 'Top9sentiment_value', 'Top10sentiment_value',
       'Top11sentiment_value', 'Top12sentiment_value', 'Top13sentiment_value',
       'Top14sentiment_value', 'Top15sentiment_value', 'Top16sentiment_value',
       'Top17sentiment_value', 'Top18sentiment_value', 'Top19sentiment_value',
       'Top20sentiment_value', 'Top21sentiment_value', 'Top22sentiment_value',
       'Top23sentiment_value', 'Top24sentiment_value', 'Top25sentiment_value']]
y = df["Label"]
print(X.shape, y.shape)

(1986, 25) (1986,)


In [44]:
#import dependencies and run train test split on the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [45]:
#set up a logistic regression classifer
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
classifier

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
#fit the training data to the model
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [47]:
#compare the score of training to testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5674949630624581
Testing Data Score: 0.48893360160965793


In [48]:
#make predictions on the testing data and compare to results of actual data. 
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [1, 1, 0, 1, 1, 0, 0, 1, 1, 0]
First 10 Actual labels: [0, 0, 1, 1, 1, 1, 1, 1, 0, 1]


In [49]:
#Display results of the testing data and the predictions
resultsDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
resultsDF["correct"] = np.where(resultsDF['Prediction'] == resultsDF["Actual"], "Yes", "No")
resultsDF

,Prediction,Actual,correct
0,1,0,No
1,1,0,No
2,0,1,No
3,1,1,Yes
4,1,1,Yes
...,...,...,...
492,1,0,No
493,1,0,No
494,1,1,Yes
495,1,0,No


In [50]:
#how many right? how many wrong? 
resultsDF = resultsDF.groupby("correct").count()
resultsDF = resultsDF["Prediction"]
resultsDF

correct
No     254
Yes    243
Name: Prediction, dtype: int64

In [51]:
#Now will try the sentiment analysis...same thing but with just daily score...
df = pd.read_csv('../Data/sentiment_values.csv')
df.head()

,Unnamed: 0,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,...,Top17sentiment_value,Top18sentiment_value,Top19sentiment_value,Top20sentiment_value,Top21sentiment_value,Top22sentiment_value,Top23sentiment_value,Top24sentiment_value,Top25sentiment_value,Daily_Sentiment
0,0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...",...,0.0,0.2,0.2,0.0,-0.8,-0.6,-0.3,-0.9,0.4,-0.30
1,1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,...,0.0,-0.7,0.0,0.0,-0.8,0.0,-0.3,-0.2,0.0,-0.08
2,2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,...,0.0,0.1,-0.6,-0.6,0.0,0.5,0.4,0.0,0.0,-0.22
3,3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",...,0.0,0.0,-0.5,-0.0,-0.3,0.5,-0.6,-0.4,-0.3,-0.15
4,4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,...,0.0,0.0,0.0,0.0,-0.4,-0.6,0.2,-0.7,0.7,-0.08


In [52]:
#time to do logistic regression
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [53]:
# Assign X (data) and y (target)
X = df[['Daily_Sentiment']]
y = df["Label"]
print(X.shape, y.shape)

(1986, 1) (1986,)


In [54]:
#import dependencies and run train test split on the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [55]:
#set up a logistic regression classifer
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
classifier

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
#fit the training data to the model
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
#compare the score of training to testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.6037609133646743
Testing Data Score: 0.5995975855130785


In [58]:
#make predictions on the testing data and compare to results of actual data. 
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [1, 1, 1, 0, 1, 0, 0, 0, 0, 0]
First 10 Actual labels: [0, 0, 1, 1, 1, 1, 1, 1, 0, 1]


In [59]:
#Display results of the testing data and the predictions
resultsDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
resultsDF["correct"] = np.where(resultsDF['Prediction'] == resultsDF["Actual"], "Yes", "No")
resultsDF

,Prediction,Actual,correct
0,1,0,No
1,1,0,No
2,1,1,Yes
3,0,1,No
4,1,1,Yes
...,...,...,...
492,1,0,No
493,1,0,No
494,0,1,No
495,1,0,No


In [60]:
#how many right? how many wrong? 
resultsDF = resultsDF.groupby("correct").count()
resultsDF = resultsDF["Prediction"]
resultsDF

correct
No     199
Yes    298
Name: Prediction, dtype: int64

In [61]:
#Try to predict the next day... 
#Now will try the sentiment analysis...same thing but with just daily score...
df = pd.read_csv('../Data/sentiment_values.csv')
df.head()

,Unnamed: 0,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,...,Top17sentiment_value,Top18sentiment_value,Top19sentiment_value,Top20sentiment_value,Top21sentiment_value,Top22sentiment_value,Top23sentiment_value,Top24sentiment_value,Top25sentiment_value,Daily_Sentiment
0,0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...",...,0.0,0.2,0.2,0.0,-0.8,-0.6,-0.3,-0.9,0.4,-0.30
1,1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,...,0.0,-0.7,0.0,0.0,-0.8,0.0,-0.3,-0.2,0.0,-0.08
2,2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,...,0.0,0.1,-0.6,-0.6,0.0,0.5,0.4,0.0,0.0,-0.22
3,3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",...,0.0,0.0,-0.5,-0.0,-0.3,0.5,-0.6,-0.4,-0.3,-0.15
4,4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,...,0.0,0.0,0.0,0.0,-0.4,-0.6,0.2,-0.7,0.7,-0.08


In [62]:
#make sure sorted by date, then move up label
df["Actual_Date"] = pd.to_datetime(df["Date"], format='%Y-%m-%d', errors='ignore')
df = df.sort_values(by="Actual_Date", ascending=True)
df["Label"] = df["Label"].shift(-1)
df.drop(df.tail(2).index,inplace=True)

In [63]:
#time to do logistic regression
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [64]:
# Assign X (data) and y (target)
X = df[['Daily_Sentiment']]
y = df["Label"]
print(X.shape, y.shape)

(1984, 1) (1984,)


In [65]:
#import dependencies and run train test split on the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [66]:
#set up a logistic regression classifer
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
classifier

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [67]:
#fit the training data to the model
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [68]:
#compare the score of training to testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5047043010752689
Testing Data Score: 0.46975806451612906


In [69]:
#make predictions on the testing data and compare to results of actual data. 
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0]
First 10 Actual labels: [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]


In [70]:
#Display results of the testing data and the predictions
resultsDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
resultsDF["correct"] = np.where(resultsDF['Prediction'] == resultsDF["Actual"], "Yes", "No")
resultsDF

,Prediction,Actual,correct
0,0.0,0.0,Yes
1,1.0,1.0,Yes
2,1.0,0.0,No
3,1.0,0.0,No
4,0.0,1.0,No
...,...,...,...
491,0.0,0.0,Yes
492,1.0,0.0,No
493,0.0,0.0,Yes
494,0.0,0.0,Yes


In [71]:
#how many right? how many wrong? 
resultsDF = resultsDF.groupby("correct").count()
resultsDF = resultsDF["Prediction"]
resultsDF

correct
No     263
Yes    233
Name: Prediction, dtype: int64

In [1]:
#This part we will try to run logistic analysis again, this time on the nmf_topic
import pandas_datareader as web
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date

In [15]:
#First we bring in the combined data set - which has the scores of each headline that the model generated. 
df = pd.read_csv('../Data/Combined_News_DJIA_single_topic.csv')
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top21,Top22,Top23,Top24,Top25,combined_headlines,lda_topic,lda_topic_readable,nmf_topic,nmf_topic_readable
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",Georgia 'downs two Russian warplanes' as coun...,1,"0.006*""russian"" + 0.006*""russia"" + 0.004*""forc...",9,"['military', 'bush', 'conflict', 'south', 'rus..."
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",Why wont America and Nato help us? If they wo...,1,"0.006*""russian"" + 0.006*""russia"" + 0.004*""forc...",9,"['military', 'bush', 'conflict', 'south', 'rus..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",Remember that adorable 9-year-old who sang at...,1,"0.006*""russian"" + 0.006*""russia"" + 0.004*""forc...",9,"['military', 'bush', 'conflict', 'south', 'rus..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,U.S. refuses Israel weapons to attack Iran: ...,1,"0.006*""russian"" + 0.006*""russia"" + 0.004*""forc...",9,"['military', 'bush', 'conflict', 'south', 'rus..."
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Saudi Arabia: Mother moves to block child ma...,b'Taliban wages war on humanitarian aid workers',"b'Russia: World ""can forget about"" Georgia\'s...",b'Darfur re

In [6]:
#time to do logistic regression
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [7]:
# Assign X (data) and y (target)
X = df[["nmf_topic"]]
y = df["Label"]
print(X.shape, y.shape)

(1986, 1) (1986,)


In [8]:
#import dependencies and run train test split on the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
#set up a logistic regression classifer
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight="balanced")
classifier

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
#fit the training data to the model
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
#compare the score of training to testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5050369375419744
Testing Data Score: 0.4768611670020121


In [12]:
#make predictions on the testing data and compare to results of actual data. 
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [1, 1, 0, 1, 1, 0, 0, 1, 1, 0]
First 10 Actual labels: [0, 0, 1, 1, 1, 1, 1, 1, 0, 1]


In [13]:
#Display results of the testing data and the predictions
resultsDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
resultsDF["correct"] = np.where(resultsDF['Prediction'] == resultsDF["Actual"], "Yes", "No")
resultsDF

,Prediction,Actual,correct
0,1,0,No
1,1,0,No
2,0,1,No
3,1,1,Yes
4,1,1,Yes
...,...,...,...
492,1,0,No
493,1,0,No
494,1,1,Yes
495,0,0,Yes


In [14]:
#how many right? how many wrong? 
resultsDF = resultsDF.groupby("correct").count()
resultsDF = resultsDF["Prediction"]
resultsDF

correct
No     260
Yes    237
Name: Prediction, dtype: int64